In [ ]:
# Add the parent directory to the path
import sys, os
sys.path.insert(0, os.path.abspath("../.."))

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt

import torch
import random
from tabulate import tabulate

from src.data.config import HP

from src.solvers.spnia_asym import AsymmetricSPNI
from src.models.ShortestPathGrb import shortestPathGrb
from src.models.ShortestPathGrid import ShortestPathGrid
from src.solvers.BendersDecomposition import BendersDecomposition

from scripts.compare_po_spo import compare_po_spo
from scripts.setup import gen_test_data, gen_train_data, setup_po_model, setup_spo_model

## Set Parameters

In [ ]:
# Define hyperparameters
c_min: float = 1.0
c_max: float = 10.0
d_min: float = 1.0
d_max: float = 10.0
Q = 0.6
B = 5
network = (6, 8)
random_seed = 31

# ML hyperparameters
num_features = 5
num_data_samples = 1000
test_size = 0.2
data_loader_batch_size = 32
po_epochs = 35
spo_epochs = 10
po_lr = .01
spo_lr = .001
sim_data_samples = 100 # number of training data
deg = 7
noise_width = 0.05

# Benders decomposition parameters
benders_max_count = 20
benders_eps = 1e-3

# Set the normalization parameter as a function of the degree
normalization_constant = {3:10, 5:20, 7:70}[deg]

# Set the random seed for reproducibility
np.random.seed(random_seed)
random.seed(random_seed)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

# Initialize the configuration class
cfg = HP()

# Change configuration parameters (optional)
cfg.set("c_min", c_min)
cfg.set("c_max", c_max)
cfg.set("d_min", d_min)
cfg.set("d_max", d_max)
cfg.set("Q", Q)
cfg.set("B", B)
cfg.set("grid_size", network)
cfg.set("random_seed", random_seed)

cfg.set("num_features", num_features)
cfg.set("num_data_samples", num_data_samples)
cfg.set("test_size", test_size)
cfg.set("data_loader_batch_size", data_loader_batch_size)
cfg.set("po_epochs", po_epochs)
cfg.set("spo_epochs", spo_epochs)
cfg.set("po_lr", po_lr)
cfg.set("spo_lr", spo_lr)
cfg.set("sim_data_samples", sim_data_samples)
cfg.set("deg", deg)
cfg.set("noise_width", noise_width)
cfg.set("normalization_constant", normalization_constant)

cfg.set("benders_max_count", benders_max_count)
cfg.set("benders_eps", benders_eps)


## Generate Network Data
Includes the network graph, a shortest path opt_model, and training data.

In [ ]:
# Define a graph with appropriate dimensions and an opt_model 
# for solving the shortest path problem on the graph
m,n = cfg.get("grid_size")
graph = ShortestPathGrid(m,n)
opt_model = shortestPathGrb(graph)

# Generate training data
training_data = gen_train_data(cfg, opt_model)

## Predict-then-Optimize

In [ ]:
po_model = setup_po_model(
    cfg,
    graph,
    opt_model,
    training_data,
    versatile=True
)

## Smart Predict-then-Optimize with OptNet

In [ ]:
spo_model = setup_spo_model(
    cfg,
    graph,
    opt_model,
    training_data,
    versatile=True
)

## Compare PO and SPO
Comparison of performance for solving shortest path problems

In [ ]:
test_data = gen_test_data(cfg)
true_objs, po_objs, spo_objs = compare_po_spo(cfg, opt_model, po_model, spo_model, test_data)

In [ ]:
po_val = [(po - true) / true for po, true in zip(po_objs, true_objs)]
spo_val = [(spo - true) / true for spo, true in zip(spo_objs, true_objs)]

# plt.scatter(np.ones_like(po_objs), po_val)
# plt.scatter(np.ones_like(spo_objs) + 1, spo_val)

fig, ax = plt.subplots(figsize=(6,4))

# Two groups: PO and SPO
_ = ax.boxplot([po_val, spo_val], tick_labels=['PO', 'SPO'],
           showmeans=True, meanline=True)   # optional styling
# plt.legend(['PO', 'SPO'], location='north')

po_mean  = np.mean(po_val)
spo_mean = np.mean(spo_val)

# std devs
po_std_pop  = np.std(po_val, ddof=0)   # population σ
po_std_samp = np.std(po_val, ddof=1)   # sample s (n-1)
spo_std_pop  = np.std(spo_val, ddof=0)
spo_std_samp = np.std(spo_val, ddof=1)
mean_improvement = (po_mean - spo_mean) * 100 / po_mean
std_improvement = (po_std_samp - spo_std_samp) * 100 / po_std_samp

rows = [
    ["Mean",         f"{po_mean*100:.2f}%",  f"{spo_mean*100:.2f}%", f"{mean_improvement:.2f}%"],
    ["Std (sample)", f"{po_std_samp*100:.2f}%", f"{spo_std_samp*100:.2f}%", f"{std_improvement:.2f}%"],
]
print(tabulate(rows, headers=["Metric", "PO (%)", "SPO (%)", "SPO improvement (%)"], tablefmt="github"))

## Compare Different Interdictors
We train different prediction models for the interdictors and compare their performance

In [ ]:
interdictions = gen_test_data(cfg)

### Naive Interdictor
This interdictor assumes that the evader has full information.

In [ ]:
# Get the number of simulation data samples
sim_data_samples = cfg.get("sim_data_samples")

# Prepare lists to store results
true_objs = []
po_objs = []
spo_objs = []

# Print that the simulation is starting
print(f"Running simulation with {sim_data_samples} samples...")

# Iterate through each data sample
for i in range(sim_data_samples):
    # Store values for the current sample
    feature = test_data["features"][i]
    cost = test_data["costs"][i]
    interdiction = interdictions["costs"][i]
    m, n = cfg.get("grid_size")
    
    # Update opt_model
    true_graph = ShortestPathGrid(m, n, cost=cost)
    opt_model.setObj(cost)

    # Update the estimated costs
    est_cost = po_model(torch.tensor(feature, dtype=torch.float32)).detach().numpy()
    spo_cost = spo_model(torch.tensor(feature, dtype=torch.float32)).detach().numpy()

    # Solutions without information asymmetry
    interdictor_I = BendersDecomposition(opt_model, k=B, interdiction_cost=interdiction, max_cnt=cfg.get("benders_max_count"), eps=cfg.get("benders_eps"))
    x_true_I, y_true_I, obj_true_I = interdictor_I.solve(versatile=False)

    # True shortest path after interdiction
    true_shortest_path = shortestPathGrb(true_graph)
    true_shortest_path.setObj(cost + x_true_I * interdiction)
    y_true_I, obj_true_I = true_shortest_path.solve()

    # PO estimated shortest path after interdiction
    est_shortest_path_po = shortestPathGrb(true_graph)
    est_shortest_path_po.setObj(est_cost + x_true_I * interdiction)
    y_est_I, obj_est_I = est_shortest_path_po.solve()

    # SPO estimated shortest path after interdiction
    est_shortest_path_spo = shortestPathGrb(true_graph)
    est_shortest_path_spo.setObj(spo_cost + x_true_I * interdiction)
    y_est_spo_I, obj_est_spo_I = est_shortest_path_spo.solve()

    # Store the results
    true_objs.append(obj_true_I)
    po_objs.append(true_graph(y_est_I))
    spo_objs.append(true_graph(y_est_spo_I))

    step = max(1, sim_data_samples // 20)  # ~every 5% (safe for small N)
    if (i % step == 0) or (i == sim_data_samples - 1):
        if i == sim_data_samples - 1:
            done, pct = 20, 100                      # final tick
        else:
            done = (i * 20) // sim_data_samples      # 0..20 “=”
            pct  = done * 5                          # 0,5,10,...,95

        sys.stdout.write('\r[%-20s] %3d%%' % ('=' * done, pct))
        sys.stdout.flush()
        if i == sim_data_samples - 1:
            sys.stdout.write('\n')


# Evaluate performance
spni_benders = {
    "true_objective": true_objs,
    "po_objective": po_objs,
    "spo_objective": spo_objs,
}

### Knowledgeable Interdictor

In [ ]:
# TODO: Run for each model (e.g., PO, SPO)
pred_model = po_model

# Get the number of simulation data samples
sim_data_samples = cfg.get("sim_data_samples")

# Print that the simulation is starting
print(f"Running simulation with {sim_data_samples} samples...")

# Prepare lists to store results
true_objs = []
est_objs = []

# Iterate through each data sample
for i in range(cfg.get("sim_data_samples")):
    # Store values for the current sample
    feature = test_data["features"][i]
    cost = test_data["costs"][i]
    interdiction = interdictions["costs"][i]
    m, n = cfg.get("grid_size")
    
    # Update opt_model
    true_graph = ShortestPathGrid(m, n, cost=cost)
    opt_model.setObj(cost)

    # Update the estimated costs
    est_cost = pred_model(torch.tensor(feature, dtype=torch.float32)).detach().numpy()

    # Solutions with information asymmetry
    asym_interdictor = AsymmetricSPNI(true_graph, budget=B, true_costs=cost, true_delays=interdiction, est_costs=est_cost, est_delays=interdiction, lsd=1)
    x_true, obj_true = asym_interdictor.solve()

    # True shortest path after interdiction
    true_shortest_path = shortestPathGrb(true_graph)
    true_shortest_path.setObj(cost + x_true * interdiction)
    y_true, obj_true = true_shortest_path.solve()

    # Estimated shortest path after interdiction
    est_shortest_path = shortestPathGrb(true_graph)
    est_shortest_path.setObj(est_cost + x_true * interdiction)
    y_est, obj_est = est_shortest_path.solve()

    # Store the results
    true_objs.append(obj_true)
    est_objs.append(true_graph(y_est))

    step = max(1, sim_data_samples // 20)  # ~every 5% (safe for small N)
    if (i % step == 0) or (i == sim_data_samples - 1):
        if i == sim_data_samples - 1:
            done, pct = 20, 100                      # final tick
        else:
            done = (i * 20) // sim_data_samples      # 0..20 “=”
            pct  = done * 5                          # 0,5,10,...,95

        sys.stdout.write('\r[%-20s] %3d%%' % ('=' * done, pct))
        sys.stdout.flush()
        if i == sim_data_samples - 1:
            sys.stdout.write('\n')

# Evaluate performance
spni_benders = {
    "true_objective": true_objs,
    "estimated_objective": est_objs,
}